# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [20]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import gmaps
import gmaps.datasets
from urllib.parse import urlencode

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key) 
file_to_load = ('output_data/cities.csv')
url1 ="https://maps.googleapis.com/maps/api/place/nearbysearch/json?location="
url2 = f"&radius=5000&type=lodging&keyword=hotel&key={g_key}"

# Read Purchasing File and store into Pandas data frame
weather_df = pd.read_csv(file_to_load)

In [2]:
#idk what that column was
weather_df = weather_df.drop(columns=['Unnamed: 0'])

In [3]:
weather_df

,City,Temperature,Humidity,Cloudiness,Wind Speed,Latitude,Longitude
0,Puerto Ayora,295.38,61,20,6.20,-0.74,-90.35
1,Bāglung,292.83,76,18,1.09,28.27,83.59
2,New Norfolk,280.37,87,35,0.45,-42.78,147.06
3,Kalutara,299.15,94,40,1.50,6.58,79.96
4,Udachny,272.15,100,90,7.00,66.42,112.40
...,...,...,...,...,...,...,...
495,Virginia Beach,300.20,58,20,5.70,36.85,-75.98
496,San Francisco del Mar Viejo,302.22,73,9,3.05,16.23,-94.65
497,Balingasay,301.52,75,100,2.62,16.36,119.86
498,Biak,296.77,80,41,2.10,-0.91,122.88


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [21]:
locations = weather_df[["Latitude", "Longitude"]]
humidity = weather_df["Humidity"]
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=np.max(humidity), point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
#compile ideal conditions
hotel_df = weather_df.loc[(weather_df["Wind Speed"] <= 5) & (weather_df["Cloudiness"] <= 80) &  (weather_df["Temperature"] >= 292) & (weather_df["Temperature"] <= 300)]

In [6]:
#new column w no values
hotel_df=weather_df
hotel_df['Hotel Name']=''

In [13]:
hotel_df.dtypes

In [14]:
#don't know if this was necessary
hotel_df["Latitude"] = hotel_df["Latitude"].astype(str)
hotel_df["Longitude"] = hotel_df["Longitude"].astype(str)

In [16]:
#loop through reponse to append hotel name results to hotel_df
for index, row in hotel_df.iterrows():
    response = requests.get(url1 + row["Latitude"] + "," + row["Longitude"] + url2).json()
    try:
        hotel_df.loc[index, "Hotel Name"] = response['results'][0]["name"]
    except:
        hotel_df.loc[index, "Hotel Name"] = "None"

In [19]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [24]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))